In [1]:
# Convert image stacks to zarr
import pathlib

import zarr
from ml4paleo.volume_providers import ImageStackVolumeProvider
import tqdm
from numcodecs import Blosc

In [5]:
image_path = pathlib.Path(
    "/Users/mateljk1/Documents/resources/data/paleo/Pterosaur slices/Araripesaurus"
)
zarr_path = pathlib.Path("../data/Araripesaurus.zarr")
image_stack = ImageStackVolumeProvider(image_path, "**/*.jp2")


In [6]:
image_stack[:,:,0].dtype, image_stack.shape

(dtype('uint16'), (3048, 2881, 9022))

In [8]:
new_zarr_file = zarr.open(
    str(zarr_path),
    mode="w",
    shape=image_stack.shape,
    chunks=(64, 64, 64),
    dtype="uint16",
    compressor=Blosc()
)

Z_CHUNK = 16
BACKGROUND_THRESHOLD = 6000

for z_start in tqdm.tqdm(range(0, image_stack.shape[2], Z_CHUNK)):
    stack = image_stack[:, :, z_start : z_start + Z_CHUNK]
    stack[stack < BACKGROUND_THRESHOLD] = 0
    new_zarr_file[:, :, z_start : z_start + Z_CHUNK] = stack


  8%|▊         | 46/564 [14:58<2:48:40, 19.54s/it]


KeyboardInterrupt: 